# Introduction to oTree!

oTreeを使って社会科学実験を作ろう！

oTree: http://www.otree.org/  
oTree documentation: https://otree.readthedocs.io/en/latest/index.html  
repo: https://github.com/myuuuuun/oymsemi_2018_camp

## やること
1. （必要があれば）Pythonをインストールする
1. oTreeをインストールする
1. Tutorial (https://otree.readthedocs.io/en/latest/tutorial/intro.html) のPart 0: Simple surveyを作る
1. Conceptual overview (https://otree.readthedocs.io/en/latest/conceptual_overview.html) を読みながらoTreeの仕組みの解説
1. 1財のケースのsecond price auctionを作る
1. （時間があれば）2財のケースのVCGオークションを作る
1. サーバにアップロードして実際の実験をする手順を確認する

## 1. Pythonのインストール

何らかの方法でPython3をインストールする. Anaconda (https://www.anaconda.com/download) がおすすめ

## 2. oTreeをインストールする

Docを参照: https://otree.readthedocs.io/en/latest/install.html

* `otree startproject oTree` を実行した後, When it asks you “Include sample games?” と聞かれたら, **Yes**を選ぶ
* インストールが終わったら, `otree devserver` を実行し, Demo画面にあるゲームを試してみる

## 3. Simple surveyを作る

Tutorial (https://otree.readthedocs.io/en/latest/tutorial/intro.html) のPart 0を参照

## 4. Conceptual overviewを読みながらoTreeの仕組みの解説

### 各実験毎のディレクトリ(例: `public_goods_simple`)の中身について

最低でも

* `models.py`
* `pages.py`
* `templates/[experiment name]/xxx.html`

の3つが必要. 以下詳細

* `models.py`: ゲームの定義を書くscript. 1組あたりのプレイヤーの人数や総ラウンド数, 各プレイヤーに入力してもらうフォームや利得の設定方法などを記述する
* `pages.py`: HTMLの表示を制御するスクリプト. 各ページ毎にclassを定義し, 対応するHTMLを`templates/[experiment name]`以下に入れることでページが表示される. ページの表示順は`page_sequence`で定義する
* `templates/[experiment name]/xxx.html`: 実験で表示するHTMLファイル. `pages.py`の中でHTMLに渡す変数を定義することで, その変数を埋め込んだHTMLを表示できる

注意点: 
* 実験結果のcsvに保存されるのは, （`round_number`や`payoff`を除いて）`models.xxxField`にセットした値のみ. どんな値が保存されているかは実験監督画面の`Data`のタブで確認できる. 独自の変数(例: signalやprivate valueを保存する必要がある場合は, Fieldを作って値をセットする必要がある)

## 5. 1財のケースのsecond price auctionを作る

実はもうある(sample gamesの中の`vickrey_auction`がそれ). 

`vickrey_auction`のディレクトリを複製して, その中のファイルをいろいろ書き換えてみる. 複製する際は, まず以下を実行する
1. 複製したディレクトリの名前を適当に変更する(以下, `second_price_auction`という名前に変更したものとする). また`static`, `templates`の中にあるディレクトリも同様に名前を変更する
1. `models.py` を開き, `class Constants`の中の`vickrey_auction`という文字列(2箇所)を両方`second_price_auction`に変更する: 
    ```py
    class Constants(BaseConstants):
        name_in_url = 'second_price_auction'
        players_per_group = 3
        num_rounds = 1

        instructions_template = 'second_price_auction/Instructions.html'
        endowment = c(100)
    
    ```
1. ルートディレクトリの下の`settings.py`を開き, `SESSION_CONFIGS`のリストに以下を追加する
```py
    {
        "name": "second_price_auction",
        "display_name": "Second Price Auction",
        "num_demo_participants": 3,
        "app_sequence": ["second_price_auction"],
    },
```
1. ついでに`settings.py`の中の`LANGUAGE_CODE`を`ja`にする
1. ターミナル（コマンドプロンプト）を開いて`control + C`でoTreeの実行を止めた後, 再度`otree devserver`を実行する. Demo画面で正常にゲームが実行できればOK

次に実験の中身をいろいろいじってみる. 例えば:

### 5.1. HTMLを全体的に日本語化してみる
1. `models.py`を開き, `class Constants`の定義に`num_other_players = players_per_group - 1`を追加する
1. `templates/vickrey_auction/Instructions.html` を開き, 英語を日本語にする. 例えば以下のようにする 
    <hr>

    ```html
    {% load otree static %}

    <div class="card bg-light instructions">
        <div class="card-body">

        <p>
            あなたにはこれからランダムに他の {{ Constants.num_other_players }} 人の被験者とグループになり, 1つしかないある商品Aのオークションに参加していただきます.
            あなたが誰とグループになったかは公開されません. <strong>全員の所持金はそれぞれ {{ Constants.endowment }} とします. </strong>
        </p>
        <p>
            入札する前に, あなたはあなたにとってのAの価値を知ることができます. Aの価値は被験者ごとに異なり, 0から{{ Constants.endowment }}までの整数の中からランダムに, 被験者ごとに独立に選ばれます. 他の被験者にとってのAの価値はあなたには知らされず, あなたにとってのAの価値も他の被験者には知らされません.
        </p>
        <p>
            次にあなたはAに対する入札額を決めます. 入札額は0から{{ Constants.endowment }}の範囲の整数で, 他の被験者には公開されません. 
            <strong>商品Aは最も入札額が高かった人が落札し, その人は2番目に高かった入札額を支払います.</strong> 最も高い額を入札した人が複数いた場合は, 落札者をランダムに決定し, 落札した人だけがその最も高い入札額を支払います. 
        </p>
        <p>
            あなた及び他の被験者の利得は, 落札した場合
        </p>
        <p style="text-align: center;">
            <strong>
                (落札した人の利得) = ({{ Constants.endowment }}) + (あなたにとってのAの価値) - (支払額, すなわち2番目に高い入札額)
            </strong>
        </p>
        <p>
            となり, 落札しなかった場合は {{ Constants.endowment }} となります. 
        </p>
        <p>
            この説明は実験中常にページ下部に表示されます.
        </p>
        </div>
    </div>
    ```
    <hr>

1. `Introduction.html`を開き, `block title`の中身を日本語にする. 例: `実験の説明`
1. `Bid.html`の中身を日本語化する. 例えば: 
    <hr>
    ```html
    {% extends "global/Page.html" %}
    {% load otree static %}

    {% block title %}
        入札額を決めてください
    {% endblock %}

    (略)

    {% block content %}

        <p>
            あなたにとっての商品Aの価値は <strong>{{ player.private_value }}</strong> です.
            もしあなたが落札した場合, あなたは
        </p>
        <p style="text-align: center;">
            <strong>{{ endowment_plus_private_value }} - (2番目に高い入札額)</strong>
        </p>
        <p>
            の利得を得ます.
        </p>

        <div class="form-group required">
            <label class="control-label" for="id_bid_amount">
                商品Aに対する入札額を決めてください (0から{{ Constants.endowment }}):</label>
            <div class="controls">

                <input type="hidden" name="bid_amount" id="id_bid_amount" value="0">
                <div class="bid-slider"></div>
                <div class="bid">あなたの入札額は: <strong><span class="bid-value"></span></strong></div>
            </div>
        </div>

        {% next_button %}

        {% include Constants.instructions_template %}

    {% endblock %}

    (略)

    ```
    <hr>
    
1. Results.htmlの中身を日本語化する. 例えば: 
    <hr>
    ```html 
    {% extends "global/Page.html" %}
    {% load otree static %}

    {% block title %}
        結果
    {% endblock %}

    {% block content %}

        <p>
            {% if player.is_winner %}
                あなたは商品Aを <span style="color: red; font-weight: bold;">落札しました！</span>
            {% else %}
                あなたは商品Aを <span style="color: green; font-weight: bold;">落札できませんでした！</span>
            {% endif %}
        </p>

        <table class="table">
            <tr>
                <th>あなたにとってのAの価値</th>
                <th>あなたの入札額</th>
                <th>最高入札額</th>
                <th>2番目に高い入札額</th>
                <th>あなたの支払額</th>
                <th>あなたの利得</th>
            </tr>

            <tr>
                <td>{{ player.private_value }}</td>
                <td>{{ player.bid_amount }}</td>
                <td>{{ group.highest_bid }}</td>
                <td>{{ group.second_highest_bid }}</td>
                <td>
                    {% if player.is_winner %}
                        {{ group.second_highest_bid }}
                    {% else %}
                        0ポイント
                    {% endif %}
                </td>
                <td>{{ player.payoff }}</td>
            </tr>

        </table>

        {% next_button %}

        {% include Constants.instructions_template %}

    {% endblock %}

    ```
    <hr>
1. Demo画面からゲームを実行してみて, エラーがでないことを確認する

### 5.2. `Bid` の画面のスライダーを, 数値を入力するフォームに変えてみる
1. `templates/vickrey_auction/Bid.html`の中の`{% block scripts %} 〜 {% endblock %}`を全てコメントアウト(`<!--`と`-->`で囲む) する
1. `<div class="controls">〜</div>`の中身も同様にコメントアウトし, かわりに`{% formfield player.bid_amount label="あなたの入札額は" %}`を入れる: 
    <hr>
    ```py
    <div class="form-group required">
        <label class="control-label" for="id_bid_amount">
            商品Aに対する入札額を決めてください (0から{{ Constants.endowment }}):</label>
        <div class="controls">
            <!--
            <input type="hidden" name="bid_amount" id="id_bid_amount" value="0">
            <div class="bid-slider"></div>
            <div class="bid">あなたの入札額は: <strong><span class="bid-value"></span></strong></div>
            -->
            {% formfield player.bid_amount label="あなたの入札額は" %}
        </div>
    </div>
    ```
    <hr>
1. Demo画面からゲームを実行してみて, エラーがでないことを確認する

### 5.3 1組あたりの人数や初期保有額, bidできる値の範囲を変更してみる

1. 1組あたりの人数をn人($n \geq 2$)に変更する場合は, `models.py`の`class Constants`の中, `players_per_group`の値を変更する
1. 初期保有額を変更するには, 同じく`class Constants`の中, `endowment`を変更する
1. bid可能な額の範囲を変更するには, `class Players`の中, `bid_amount`の`min`と`max`を変更すれば良い. 初期値では最小値が0, 最大値が`Constants.endowment`の値となっている

### 5.4 疑似乱数のseedを設定する

乱数の再現性を担保するため. 

1. `models.py`を開く
1. 最初の行に `import numpy as np` を追加する
1. `class Constants`に以下を追加する
    ```py
    seed = 20180921
    ```
1. `class Subsession`のメソッド`creating_session`を以下のように変更する
    ```py
    def creating_session(self):
        self.session.vars["random_state"] = np.random.RandomState(Constants.seed)
        for p in self.get_players():
            p.private_value = self.session.vars["random_state"].randint(0, Constants.endowment)
    ```
1. `class Group`のメソッド`set_payoffs`の中, `winner = random.choice(players_with_highest_bid)`を
    ```py
    winner = self.session.vars["random_state"].choice(players_with_highest_bid)
    ```
    に変更する
1. Demo画面からゲームを実行してみて, エラーがでないことを確認する
1. （時間があれば）seedが同じなら, 各個人のprivate valueや, 複数人が同じbidをした時に落札する人は常に同じであることを確かめる. またseedの値を変更するとそれらが変わることを確かめる

### 5.5 実験を複数回繰り返すようにする

1. 単純には `models.py` > `class Constants` > `num_rounds` の値を変更すればOK(例えば`num_rounds = 3`とする). ただしこのままだとIntroductionが複数回表示されてしまうので, 次の処理をする
1. `pages.py`を開き, `class Introduction` を以下のように書き換える
    ```py
    class Introduction(Page):
        def is_displayed(self):
            return self.round_number == 1
    ```
    こうすることで, Introductionが最初の1回だけ表示されるようになる
1. また, このままだと各個人のprivate valueが全ての実験で同じ値になってしまう. これを回避するためには, `models.py` > `class Subsession` > `creating_session` を以下のように変更する
    ```py
    def creating_session(self):
        if self.round_number == 1:
            self.session.vars["random_state"] = np.random.RandomState(Constants.seed)
        
        for p in self.get_players():
            p.private_value = self.session.vars["random_state"].randint(0, Constants.endowment)
    ```
1. さらに最終結果ページを作る. まず`Results.html`をコピーして`FinalResults.html`を作り, 中身を以下のように変更する
    <hr>
    ```html
    {% extends "global/Page.html" %}
    {% load otree static %}

    {% block title %}
        最終結果
    {% endblock %}

    {% block content %}

        <p>以上で全ての実験は終了です。お疲れ様でした。以下が最終結果です。</p>

        <table class="table">
            <tr>
                <th>期</th>
                <th>あなたにとってのAの価値</th>
                <th>あなたの入札額</th>
                <th>最高入札額</th>
                <th>2番目に高い入札額</th>
                <th>あなたが落札したか</th>
                <th>利得</th>
            </tr>

            {% for player in player_in_all_rounds %}
            <tr>
                <td>{{ player.round_number }}</td>
                <td>{{ player.private_value }}</td>
                <td>{{ player.bid_amount }}</td>
                <td>{{ player.group.highest_bid }}</td>
                <td>{{ player.group.second_highest_bid }}</td>
                <td>{{ player.is_winner }}</td>
                <td>{{ player.payoff }}</td>
            </tr>
            {% endfor %}

            <tr>
                <th colspan="6">合計</th>
                <td>{{ total_payoff }}</td>
            </tr>

        </table>

        {% include Constants.instructions_template %}

    {% endblock %}

    ```
    <hr>
    次に`pages.py`に以下のような`class FinalResults`を作る
    ```py
    class FinalResults(Page):
        def is_displayed(self):
            return self.subsession.round_number == Constants.num_rounds

        def vars_for_template(self):
            total_payoff = sum([p.payoff for p in self.player.in_all_rounds()])

            return {
                "player_in_all_rounds": self.player.in_all_rounds(),
                "total_payoff": total_payoff,
            }
    
    ```
    最後に`page_sequence`に`FinalResults`を追加する:
    ```py
    page_sequence = [
        Introduction,
        Bid,
        ResultsWaitPage,
        Results, 
        FinalResults]
    ```
1. 最後に, きちんとFinalResultsが表示されるかを確認する

## 6. 2財のケースのVCGオークションを作る

See my github repo: https://github.com/myuuuuun/oymsemi_2018_camp

## 7. サーバにアップロードして実際の実験をする手順を確認する

この辺を見る: https://otree.readthedocs.io/en/latest/server/intro.html

実演する